# GROUP 19 DATA WRANGLING PROJECT
---

#### **TOPIC**: Marketing Strategy Effectivity
This research will be taking a look at the effects of different types of marketing campaigns and their effectivity in terms of clickthrough rate, customer engagement, and customer acquisition.

In [16]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [17]:
campaigns_df = pd.read_csv("campaigns.csv")
messages_df = pd.read_csv("messages-demo.csv")

messages_df.head()

C:\Users\Migs\AppData\Local\Temp\ipykernel_15960\1183315834.py:2: DtypeWarning: Columns (7,8,16,17,19,21,23,25,27,29) have mixed types. Specify dtype option on import or set low_memory=False.
  messages_df = pd.read_csv("messages-demo.csv")


,id,message_id,campaign_id,message_type,client_id,channel,category,platform,email_provider,stream,...,is_soft_bounced,soft_bounced_at,is_complained,complained_at,is_blocked,blocked_at,is_purchased,purchased_at,created_at,updated_at
0,3527358,3f6aaad3-bab7-4886-b083-fe8c1f210066,31,transactional,1515915625489833514,email,NaN,NaN,mail.ru,desktop,...,f,NaN,f,NaN,f,NaN,t,2021-05-06 16:40:38,2023-04-27 08:55:05.883908,2023-04-27 08:57:33.080129
1,3527619,0e670ecc-4549-44f6-86ed-469682d34837,32,transactional,1515915625489220445,email,NaN,NaN,yandex.ru,desktop,...,f,NaN,f,NaN,f,NaN,f,NaN,2023-04-27 08:55:06.265821,2023-04-27 08:56:18.60223
2,3527980,276b25cf-1bda-4faf-b3a4-98e4161f9357,32,transactional,1515915625489854185,email,NaN,NaN,mail.ru,desktop,...,f,NaN,f,NaN,f,NaN,f,NaN,2023-04-27 08:55:06.777039,2023-04-27 08:56:19.112546
3,3528369,4545aff2-09b3-45e3-9abd-c680357e5429,32,transactional,1515915625489101550,email,NaN,NaN,mail.ru,desktop,...,f,NaN,f,NaN,f,NaN,f,NaN,2023-04-27 08:55:07.325906,2023-04-27 08:56:19.590637
4,3528648,5850858d-2dcf-4f31-a0d3-5db5649b17c4,32,transactional,1515915625490455948,email,NaN,NaN,mail.ru,desktop,...,f,NaN,f,NaN,f,NaN,f,NaN,2023-04-27 08:55:07.727792,2023-04-27 08:56:19.926474


In [18]:
print(len(messages_df))
print(len(campaigns_df))

10000000
1907


### Sampling
Since the data is about 10 million rows big, the group decided to use only a sample of 40% of the data to cut down on computing time.

In [19]:
messages_df = messages_df[messages_df["message_type"] == "bulk"] # since bulk are over 93% of the campains we are only working with bulk messages and bulk campaings
campaigns_df = campaigns_df[campaigns_df["campaign_type"] == "bulk"]

messages_sample = messages_df.sample(frac= 0.40, random_state= 42, replace= False)

In [20]:
messages_sample.head()
print(len(messages_sample))

2824958


### Data Cleanup
We will eliminate all of the unneccessary columns from the dataframes we created above and consolidate them into one `merged_df`

In [21]:
print("Messages Database:\n",list(messages_df.columns),"\n")
print("Campaigns Database:\n",list(campaigns_df.columns))

# our sample has many columns >>> some are disposable and some are missing the majority of the values => these will be dropped

messages_sample = messages_sample.drop(["id", "category", "platform", "stream", "hard_bounced_at", "soft_bounced_at",
                                         "is_soft_bounced", "is_hard_bounced",  "blocked_at", "updated_at", "is_complained",
                                         "complained_at", "opened_first_time_at", "clicked_last_time_at", "unsubscribed_at", 
                                         "purchased_at", "created_at", "message_type", "message_id", "date", "email_provider",
                                         "opened_last_time_at", "clicked_first_time_at"],
                                           axis= 1)

campaigns_df = campaigns_df.drop(["ab_test", "warmup_mode", "hour_limit", "is_test", "position", "campaign_type"], axis=1)



Messages Database:
 ['id', 'message_id', 'campaign_id', 'message_type', 'client_id', 'channel', 'category', 'platform', 'email_provider', 'stream', 'date', 'sent_at', 'is_opened', 'opened_first_time_at', 'opened_last_time_at', 'is_clicked', 'clicked_first_time_at', 'clicked_last_time_at', 'is_unsubscribed', 'unsubscribed_at', 'is_hard_bounced', 'hard_bounced_at', 'is_soft_bounced', 'soft_bounced_at', 'is_complained', 'complained_at', 'is_blocked', 'blocked_at', 'is_purchased', 'purchased_at', 'created_at', 'updated_at'] 

Campaigns Database:
 ['id', 'campaign_type', 'channel', 'topic', 'started_at', 'finished_at', 'total_count', 'ab_test', 'warmup_mode', 'hour_limit', 'subject_length', 'subject_with_personalization', 'subject_with_deadline', 'subject_with_emoji', 'subject_with_bonuses', 'subject_with_discount', 'subject_with_saleout', 'is_test', 'position']


In [22]:
display(messages_sample[:10])

,campaign_id,client_id,channel,sent_at,is_opened,is_clicked,is_unsubscribed,is_blocked,is_purchased
7286261,230,1515915625488886720,email,2021-05-27 08:12:44,f,f,f,f,f
848463,64,1515915625608891382,mobile_push,2021-04-30 08:32:01,f,f,f,f,f
5778692,150,1515915625486927106,email,2021-05-21 08:58:57,f,f,f,f,f
7539782,230,1515915625488084724,email,2021-05-27 10:32:50,t,f,f,f,f
6710082,152,1515915625490799486,email,2021-05-24 08:41:10,f,f,f,f,f
827892,64,1515915625571018047,mobile_push,2021-04-30 07:58:37,f,f,f,f,f
7945106,257,1515915625490421081,mobile_push,2021-05-28 13:00:24,f,f,f,f,f
7965469,257,1515915625501578324,mobile_push,2021-05-28 13:00:33,f,f,f,f,f
6111520,150,1515915625500830959,email,2021-05-21 07:17:51,t,f,f,f,f
9701465,366,1515915625500834555,email,2021-06-10 08:30:54,f,f,f,f,f


In [23]:
display(campaigns_df[:10])

,id,channel,topic,started_at,finished_at,total_count,subject_length,subject_with_personalization,subject_with_deadline,subject_with_emoji,subject_with_bonuses,subject_with_discount,subject_with_saleout
0,63,mobile_push,sale out,2021-04-30 07:22:36.615023,2021-04-30 07:23:41,48211.0,146.0,False,False,True,False,False,False
1,64,mobile_push,sale out,2021-04-30 09:02:50.817227,2021-04-30 09:04:08,1037337.0,97.0,False,False,True,False,False,False
2,78,mobile_push,sale out,2021-05-06 07:14:10.533318,2021-05-06 07:15:17,70080.0,146.0,False,False,True,False,False,False
3,79,mobile_push,sale out,2021-05-06 09:03:56.486750,2021-05-06 09:42:15,921838.0,97.0,False,False,True,False,False,False
4,89,mobile_push,NaN,2021-05-07 11:54:06.168664,2021-05-07 11:54:38,45503.0,109.0,False,True,True,False,False,False
5,110,mobile_push,sale out,2021-05-12 07:38:32.980268,2021-05-12 07:40:16,90816.0,146.0,False,False,True,False,False,False
6,111,mobile_push,sale out,2021-05-12 07:44:41.865082,2021-05-12 08:16:07,1045217.0,97.0,False,False,True,False,False,False
7,129,email,sale out,2021-05-18 07:38:49.825780,2021-05-18 07:38:58,3.0,133.0,False,False,True,False,False,False
8,136,email,sale out,2021-05-19 07:00:11.121170,2021-05-19 08:03:19,177363.0,133.0,False,False,True,False,False,False
9,138,mobile_push,sale out,2021-05-19 07:00:11.105536,2021-05-19 07:21:42,37889.0,146.0,False,False,True,False,False,False


In [29]:
merged_df = pd.merge(messages_sample, campaigns_df, left_on = "campaign_id", right_on= "id" ,how= "inner")
print(merged_df.isna().sum())

campaign_id                         0
client_id                           0
channel_x                           0
sent_at                             0
is_opened                           0
is_clicked                          0
is_unsubscribed                     0
is_blocked                          0
is_purchased                        0
id                                  0
channel_y                           0
topic                           28757
started_at                          0
finished_at                         0
total_count                         0
subject_length                      0
subject_with_personalization        0
subject_with_deadline               0
subject_with_emoji                  0
subject_with_bonuses                0
subject_with_discount               0
subject_with_saleout                0
dtype: int64


### Converting string and bool values to binary
Since the database contains a lot of string values to represent True or False, we will convert these values to binary to have an easier time understanding the database and using it to analyze the data.

In [31]:
def convert_to_binary(value):
    value_lower = str(value).lower()
    if value_lower == 't':
        return 1
    elif value_lower == 'f':
        return 0
    else:
        return value

merged_df = merged_df.applymap(lambda x: int(x) if isinstance(x, bool) else x)
merged_df = merged_df.applymap(convert_to_binary)


In [36]:
pd.set_option('display.max_columns', None)
display(merged_df[:10]) 

,campaign_id,client_id,channel_x,sent_at,is_opened,is_clicked,is_unsubscribed,is_blocked,is_purchased,id,channel_y,topic,started_at,finished_at,total_count,subject_length,subject_with_personalization,subject_with_deadline,subject_with_emoji,subject_with_bonuses,subject_with_discount,subject_with_saleout
0,230,1515915625488886720,email,2021-05-27 08:12:44,0,0,0,0,0,230,email,sale out,2021-05-27 08:00:10.980791,2021-05-27 10:35:06,651859.0,115.0,0,0,1,0,0,0
1,230,1515915625488084724,email,2021-05-27 10:32:50,1,0,0,0,0,230,email,sale out,2021-05-27 08:00:10.980791,2021-05-27 10:35:06,651859.0,115.0,0,0,1,0,0,0
2,230,1515915625501096510,email,2021-05-27 09:19:47,0,0,0,0,0,230,email,sale out,2021-05-27 08:00:10.980791,2021-05-27 10:35:06,651859.0,115.0,0,0,1,0,0,0
3,230,1515915625502848292,email,2021-05-27 08:16:56,0,0,1,0,0,230,email,sale out,2021-05-27 08:00:10.980791,2021-05-27 10:35:06,651859.0,115.0,0,0,1,0,0,0
4,230,1515915625487737557,email,2021-05-27 09:44:49,0,0,0,0,0,230,email,sale out,2021-05-27 08:00:10.980791,2021-05-27 10:35:06,651859.0,115.0,0,0,1,0,0,0
5,230,1515915625488545980,email,2021-05-27 10:02:39,0,0,0,0,0,230,email,sale out,2021-05-27 08:00:10.980791,2021-05-27 10:35:06,651859.0,115.0,0,0,1,0,0,0
6,230,1515915625488327822,email,2021-05-27 09:57:28,0,0,0,0,0,230,email,sale out,2021-05-27 08:00:10.980791,2021-05-27 10:35:06,651859.0,115.0,0,0,1,0,0,0
7,230,1515915625489229184,email,2021-05-27 09:41:51,0,0,0,0,0,230,email,sale out,2021-05-27 08:00:10.980791,2021-05-27 10:35:06,651859.0,115.0,0,0,1,0,0,0
8,230,1515915625500283542,email,2021-05-27 10:11:37,0,0,0,0,0,230,email,sale out,2021-05-27 08:00:10.980791,2021-05-27 10:35:06,651859.0,115.0,0,0,1,0,0,0
9,230,1515915625487439364,email,2021-05-27 08:05:55,0,0,0,0,0,230,email,sale out,2021-05-27 08:00:10.980791,2021-05-27 10:35:06,651859.0,115.0,0,0,1,0,0,0


In [12]:
merged_df['channel_x'].value_counts()

mobile_push    1690477
email          1134481
Name: channel_x, dtype: int64

Need to check whether each type of `subject_type` affects `is_opened` and `is_purchased`